## Análisis del reclutamiento por campaña

In [36]:
import pandas as pd

In [71]:
# Cargo las columnas que necesito
datos = pd.read_csv('/home/lautaro/Desktop/events.csv',
                    usecols = lambda col: col in ["event", "timestamp", "new_vs_returning", "campaign_source", "person"])

In [72]:
# Filtro los eventos que me interesan
visitas = datos.loc[datos["event"] == "visited site",["timestamp", "new_vs_returning", "person"]]
campania = datos.loc[datos["event"] == "ad campaign hit",["timestamp", "campaign_source", "person"]]

In [73]:
visitas = visitas.rename(index=str, columns = {"timestamp" : "hora visita"})
campania = campania.rename(index=str, columns = {"timestamp" : "hora campania"})

In [74]:
# Obtengo las visitas de usuarios nuevos
visitas = visitas.loc[visitas["new_vs_returning"] == "New",["hora visita", "person"]]

In [75]:
# Me quedo con la primer fecha de acceso a la pagina de cada usuario ya que el resto de las veces seria un "Returning"
campania = campania.groupby("person", as_index=False).min()

In [111]:
# Matcheamos la hora de la primer visita del usuario al sitio con la hora de su primer click en campaña
visitas_por_campania = visitas.merge(campania, how="inner", left_on="person", right_on="person")
visitas_por_campania = visitas_por_campania.drop(columns="person")

In [112]:
# Convierto las columnas de fechas a datetime para poder operar
visitas_por_campania["hora visita"] = pd.to_datetime(visitas_por_campania["hora visita"])
visitas_por_campania["hora campania"] = pd.to_datetime(visitas_por_campania["hora campania"])

In [113]:
# Para contabilizarlo como nuevo debido a la campaña la diferencia de tiempo entre los eventos debe ser poca
# Establezco un límite de 5 segundos de diferencia máxima
visitas_por_campania = visitas_por_campania.loc[((visitas_por_campania["hora visita"] - visitas_por_campania["hora campania"]) < pd.Timedelta("0 days 00:00:05")) & 
                                               ((visitas_por_campania["hora visita"] - visitas_por_campania["hora campania"]) > pd.Timedelta("-1 days +23:59:55")), :]

In [114]:
visitas_por_campania = visitas_por_campania.drop(columns="hora visita")
visitas_por_campania["cantidad"] = 1

In [118]:
# Puedo analizar como aportan las campañas mes a mes
campania_por_mes = visitas_por_campania
campania_por_mes["mes"] = campania_por_mes["hora campania"].dt.month
campania_por_mes = campania_por_mes.drop(columns="hora campania")

In [120]:
campania_por_mes = campania_por_mes.groupby(["campaign_source", "mes"], as_index=False).sum()

In [124]:
print(campania_por_mes.loc[campania_por_mes["mes"] == 1,:])

   campaign_source  mes  cantidad
0              inf    1         4
11     FacebookAds    1        25
17  FacebookSocial    1         2
26          afilio    1        31
32            bing    1         7
42         buscape    1        10
45          criteo    1        83
54          emblue    1         3
60          google    1       147
82           zanox    1         1
